In [2]:
import pickle
import pandas as pd
train=pd.read_csv(r"D:\Monhoc\AI\Code\databaitapb3.csv")
test=pd.read_csv(r"D:\Monhoc\AI\Code\test.csv")
# xóa cột ID
train=train.drop(columns=['Id'])
# thay tất cả các giá trị trống của cột lotFrontage bằng giá trị trung bình
train['LotFrontage']=train['LotFrontage'].fillna(train['LotFrontage'].mean())
test['LotFrontage']=test['LotFrontage'].fillna(test['LotFrontage'].mean())
# thay tất cả các giá trị trống của 2 cột dưới đây bằng giá trị phổ biến nhất
test['MSZoning']=test['MSZoning'].fillna('RL')
test['SaleType']=test['SaleType'].fillna('WD')
print(train.isnull().sum())
print(test.isnull().sum())

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 80, dtype: int64
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64


In [3]:
#  viết hàm biến đổi tất các các cột kiểu object thành kiểu int
def convert_object_columns_to_int(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.factorize(df[col])[0]
    return df
def encoded_input_data(df):
    df['MSSubClass']=df['MSSubClass'].astype('object')
    df['YrSold']=df['YrSold'].astype('object')
    df['LotFrontage']=df['LotFrontage'].astype('float')
    convert_object_columns_to_int(df)
    return df
encoded_input_data(train)
encoded_input_data(test)
# điền tất cả các giá trị còn thiếu bằng giá trị trung bình
train.fillna(train.mean(),inplace=True)
test.fillna(train.mean(),inplace=True)


In [4]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành hai phần: 80% dữ liệu để train và 20% để test
train_data, test_data = train_test_split(train, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestRegressor

# Tách biến phụ thuộc (y) và biến độc lập (X) trong tập huấn luyện
X_train = train_data.drop(columns=['SalePrice'])
y_train = train_data['SalePrice']

# Huấn luyện mô hình Random Forest 1 
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
# Tách biến phụ thuộc (y) và biến độc lập (X) trong tập kiểm tra
X_test = test_data.drop(columns=['SalePrice'])
y_test = test_data['SalePrice']

# Đánh giá hiệu suất của mô hình 1 trên tập kiểm tra
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 884467233.5410237


In [5]:
def predictFunction(id):
    row = test.loc[test['Id'] == id]
    row=row.drop(columns=['Id'])
    lmao=rf_model.predict(row)
    return lmao
result_df = pd.DataFrame()
# Áp dụng hàm predictFunction cho mỗi giá trị Id và lưu kết quả vào DataFrame mới
result_df['Id'] = test['Id']  # Sử dụng cột Id từ DataFrame gốc
result_df['SalePrice'] = test['Id'].apply(predictFunction)
print(result_df)
result_df['SalePrice']=result_df['SalePrice'].astype(float)
result_df.to_csv('ket_qua_du_doan.csv', index=False)


        Id    SalePrice
0     1461  [134736.88]
1     1462   [158577.5]
2     1463   [185447.5]
3     1464  [194294.52]
4     1465  [216363.68]
...    ...          ...
1454  2915   [89372.25]
1455  2916    [94398.0]
1456  2917  [161352.74]
1457  2918   [115291.5]
1458  2919   [224718.1]

[1459 rows x 2 columns]


In [6]:
with open("random-forest-regressor.pkl", "wb") as f:
    pickle.dump(rf_model, f)
with open("logistic.pkl", "wb") as m:
    pickle.dump(rf_model, m)